# Violation zone

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
from scipy.optimize import least_squares


def insertCol(df):
    '''Insert empty columns to get violation info for requirements R1-R3'''
    df.insert(len(df.columns), 'R1', '')
    df.insert(len(df.columns), 'R2', '')
    df.insert(len(df.columns), 'R3', '')
    df.insert(len(df.columns), 'r1', '')
    df.insert(len(df.columns), 'r2', '')
    df.insert(len(df.columns), 'r3', '')
    df.insert(len(df.columns), 'V', '')
    return df


def checkViolation(p1,p2,p3,p4,T1,T1F,T2,T2F,TR,T3,Pretry):
    violation = 0
    r1,r2,r3 =0,0,0
    # compute R1-R3
    R1= (p1*p2*p4)/(1-Pretry*p3)
    R2= -1*(p1*(p3*T2-p2*T3-p3*T2F-T1-T2+T1F)+Pretry*p3*(T1F-p1*TR-p1*T1F)-T1F) / (1-Pretry*p3)
    R3 = (p1*p2*(1-p4))/(1-Pretry*p3)
    # check violation   (i.e., does not comply with innequalities, hence innequality symbols are flipped)
    if R1 <= R1_bound:  violation = 1; r1=1
    if R2 >= R2_bound:  violation = 1; r2=1
    if R3 >= R3_bound:  violation = 1; r3=1
    return [R1,R2,R3,r1,r2,r3,violation]


def addViolation_lookupTable(df):
    for index,row in df.iterrows():
        M1 = row['M1']; M2= row['M2']; M3 = row['M3']
        x = row['M3'] #dummy
        #check violation
        [R1,R2,R3,r1,r2,r3,violation] = checkViolation(p1(M1,M2,M3),p2(M1,M2,M3),p3(M1,M2,M3),p4(M1,M2,M3),T1(M1,M2,M3),
                                                       T1F(M1,M2,M3),T2(M1,M2,M3),T2F(M1,M2,M3),TR(M1,M2,M3),T3(M1,M2,M3),pR(M1,M2,M3))
        #add violation
        df.loc[index,'R1'],df.loc[index,'R2'],df.loc[index,'R3'] = R1,R2,R3
        df.loc[index,'r1'],df.loc[index,'r2'],df.loc[index,'r3'] = r1,r2,r3
        df.loc[index,'V'] = violation
    
    file = 'output_with_violation.csv'
    df.to_csv(file, index=False)
    print('saved in ',file)
    return df

def print_results_violation(df):
    # Sum the values in the 'Age' column
    total_violation = df['V'].sum()
    total = len(df['V'])
    print(" Total data points: {}.\n Total data points in violation: {}.\n Percentage in violation: {}%."
          .format(total,total_violation,total_violation/total*100))

def read_csv(readings_file):
    dfM = pd.read_csv(readings_file)
    # dfM.columns=["Time","M1","M2","M3","state"]
    #dfM = insertCol(dfM)
    return dfM



def plot_violation_lookupTable(df_HeatMap,title):
    # Create a 3D scatter plot
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    # Extracting columns for x, y, z coordinates
    x = df_HeatMap['M1']
    y = df_HeatMap['M2']
    z = df_HeatMap['M3']
    # Color scale based on violation
    c = df_HeatMap['V']
    # Scatter plot with color scale
    scatter = ax.scatter(x, y, z, c=c, cmap='viridis')
    # Adding colorbar
    cbar = fig.colorbar(scatter)
    cbar.set_label('violation')
    # Set labels for axes
    ax.set_xlabel('Init Light value')
    ax.set_ylabel('Init Floor friction')
    ax.set_zlabel('Init Object friction')
    # Plot
    plt.title('3D Scatter Plot with Color Scale')
    plt.show()


def plot3D_violation_lookupTable(df_HeatMap, title):
    '''Interactive 3D plot'''
    fig = go.Figure()
    # Scatter plot with color scale
    fig.add_trace(go.Scatter3d(
        x=df_HeatMap['M1'],
        y=df_HeatMap['M2'],
        z=df_HeatMap['M3'],
        mode='markers',
        marker=dict(
            size=5,
            color=df_HeatMap['V'],  # Color scale based on violation
            colorscale='Plasma',#'Viridis',  # You can choose different color scales
            opacity=0.8,
            colorbar=dict(title='Violation')  # Adding color bar with title
    )))
    # Set labels for axes and title
    fig.update_layout(
        scene=dict(
            xaxis_title='Init Light value',
            yaxis_title='Init Floor friction',
            zaxis_title='Init Object friction'
        ),
        title=title)
    fig.show()

        

Add safe-violation zone boundary and edge (out of ODD) 

In [3]:
def checkEdge(R1,R2,R3):
    '''Check if on edge of ODD (in save zone)'''
    if R1

def checkSafeBoundary():


def addBoundaryEdge_lookupTable(df):
    for index,row in df.iterrows():
        M1= row['M1']; M2= row['M2']; M3= row['M3']

        R1,R2,R3 = df.loc[index,'R1'],df.loc[index,'R2'],df.loc[index,'R3']
        
    #     x= row['M3'] #dummy
    #     # check only on safe zone
    #     if row['V']==0:
    #         #check edge
    #         [R1,R2,R3,r1,r2,r3,violation] = checkViolation(p1(M1,M2,M3),p2(M1,M2,M3),p3(M1,M2,M3),p4(M1,M2,M3),T1(M1,M2,M3),
    #                                                     T1F(M1,M2,M3),T2(M1,M2,M3),T2F(M1,M2,M3),TR(M1,M2,M3),T3(M1,M2,M3),pR(M1,M2,M3))
    #         #add violation
    #         df.loc[index,'R1'],df.loc[index,'R2'],df.loc[index,'R3'] = R1,R2,R3
    #         df.loc[index,'r1'],df.loc[index,'r2'],df.loc[index,'r3'] = r1,r2,r3
    #         df.loc[index,'V'] = violation
    
    # file = 'output_with_violation.csv'
    # df.to_csv(file, index=False)
    # print('saved in ',file)
    # return df

SyntaxError: invalid syntax (177134984.py, line 3)

### Set up problem

In [ ]:
#============ SET UP PROBLEM
# Req. bounds
R1_bound = 0.6; R2_bound = 100; R3_bound = 0.35 # % violated


#M1 value
minM1=0.5; maxM1= 1; M =20
#M2
minM2=0.25; maxM2=0.5; smoothN=20
#M3
minM3=0.5; maxM3=1; L=20
#fun
p1 = lambda M1,M2,M3: 0.2018 +  0.8191 * M1
p2 = lambda M1,M2,M3: 0.414 + 0.4612 * M1
p3 = lambda M1,M2,M3: 0.1
p4 = lambda M1,M2,M3: -0.1618+ (1.2523 * M3)
pR = lambda M1,M2,M3: 0.8
T1 = lambda M1,M2,M3: 0     
T2 = lambda M1,M2,M3: 19.765 + (15.627 * M1)
T3 = lambda M1,M2,M3: 63.15                          
T1F= lambda M1,M2,M3: 0
T2F= lambda M1,M2,M3: 7.343 + ( 72.234 * M1) + (161.485 * M2) - (231.337 * M1*M2)
TR = lambda M1,M2,M3: 6.498 - (5.482 * M1) - (6.855 * M3) + (8.301 * M1*M3)


# boundary (max min values of requirements values)
bound_epsilon = 0.001
# boundary expressions
R1_boundary = lambda R1_value: (R1_bound + bound_epsilon >= R1_value) & (R1_value >= R1_bound)
R2_boundary = lambda R1_value: R1_value == R1_bound
R3_boundary = lambda R1_value: R1_value == R1_bound


#edge (out of ODD)
M1_edge = lambda M1_value: M1_value>= maxM1 or M1_value<=minM1
M2_edge = lambda M2_value: M2_value>= maxM1 or M2_value<=minM1
M3_edge = lambda M3_value: M3_value>= maxM1 or M3_value<=minM1



#file
hist_file = "./output_julie.csv"


### Run

In [ ]:
#=============== RUN
df= read_csv(hist_file)
df = addViolation_lookupTable(df)
print_results_violation(df)

# Plot
plot3D_violation_lookupTable(df,'M1, M2, M3 and violation over Time')

saved in  output_with_violation.csv
 Total data points: 216.
 Total data points in violation: 168.0.
 Percentage in violation: 77.77777777777779%.


In the last figure, the floor friction does not affect the violation. This is a problem as we would like all M1,M2,M3 to have an effect on the violation of requirements.

There is also some weird violations happening when light= 1. Intuitivelly, more light should result in lower violations...



In [ ]:
df

,M1,M2,M3,P1,T1,T1F,P2,P3,T2,TR,T2F,P4,T3,R1,R2,R3,r1,r2,r3,V
0,1.0,0.50,0.5,1.0,6.2,NaN,0.8,0.0,35.599998,NaN,NaN,0.00,52.250000,0.450971,101.785838,0.520216,1.0,1.0,1.0,1.0
1,0.8,0.50,0.5,1.0,6.2,NaN,1.0,0.0,35.599998,NaN,NaN,0.00,57.000000,0.338703,78.234708,0.390710,1.0,0.0,1.0,1.0
2,0.5,0.30,0.9,0.2,8.0,49.0,1.0,0.0,29.000000,NaN,NaN,1.00,60.000000,0.413467,47.415157,0.014876,1.0,0.0,0.0,1.0
3,0.9,0.50,0.7,1.0,7.4,NaN,0.8,0.0,31.400000,NaN,NaN,1.00,53.500000,0.604867,89.675426,0.241326,0.0,0.0,0.0,0.0
4,0.8,0.45,0.6,1.0,7.4,NaN,1.0,0.0,30.600000,NaN,NaN,0.80,55.400002,0.430047,78.342969,0.299365,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0.9,0.25,0.5,1.0,7.2,NaN,1.0,0.0,34.400002,NaN,NaN,0.00,72.599998,0.392930,90.857434,0.453263,1.0,0.0,1.0,1.0
212,0.6,0.35,0.5,1.0,6.2,NaN,1.0,0.0,38.599998,NaN,NaN,0.20,58.400002,0.241688,57.186425,0.278799,1.0,0.0,0.0,1.0
213,0.6,0.40,0.7,1.0,6.4,NaN,0.8,0.2,32.500000,NaN,58.0,0.75,58.000000,0.372050,57.249288,0.148438,1.0,0.0,0.0,1.0
214,0.8,0.45,0.7,1.0,6.6,NaN,0.8,0.0,36.000000,NaN,NaN,1.00,55.250000,0.521391,78.341373,0.208021,1.0,0.0,0.0,1.0


In [ ]:
df[df['M1']==1]

NameError: name 'df' is not defined